In [1]:
import numpy as np
import glob, os
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
dir = './sequence/*'

In [3]:
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [5]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 100

In [21]:
# timesteps 맞춰주기 위해서 padding 추가
#all_data = sequence.pad_sequences(all_data, padding='post', maxlen = max_sequence_length)

In [6]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(None, n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(n_features, return_sequences=True))
#model.add(Dense(1))
#model.add(LSTM(n_features, return_sequences=True))
#model.add(LSTM(10, return_sequences=True))


W0914 16:17:27.291615 4689302976 deprecation_wrapper.py:119] From //anaconda3/envs/graph/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0914 16:17:27.352676 4689302976 deprecation_wrapper.py:119] From //anaconda3/envs/graph/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 16:17:27.356863 4689302976 deprecation_wrapper.py:119] From //anaconda3/envs/graph/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [7]:
model.compile(loss='mean_squared_error', optimizer='adam')

W0914 16:17:30.820363 4689302976 deprecation_wrapper.py:119] From //anaconda3/envs/graph/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [13]:
import random as rd
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0
def val_generator(x_val):
    idx = 0
    while True:
        yield np.array([x_val[idx]]), np.array([x_val[idx]])
        idx += 1
        if idx >= len(x_val):
            idx = 0
model.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=540, verbose=1)#, validation_data=val_generator(x_val))

Epoch 1/100
540/540 [==============================] - 117s 217ms/step - loss: 16.7534
Epoch 2/100
540/540 [==============================] - 113s 208ms/step - loss: 16.7532
Epoch 3/100
540/540 [==============================] - 84s 156ms/step - loss: 16.7532
Epoch 4/100
540/540 [==============================] - 83s 154ms/step - loss: 16.7532
Epoch 5/100
396/540 [=====================>........] - ETA: 25s - loss: 16.6634

KeyboardInterrupt: 

In [ ]:
model.summary()